# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686589


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:11,  2.57s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:59,  2.22s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:39,  1.57s/it]

Rendering models:  23%|██▎       | 7/30 [00:06<00:25,  1.11s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:07<00:01,  6.45it/s]

Rendering models:  83%|████████▎ | 25/30 [00:07<00:00,  7.48it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  8.42it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00,  9.64it/s]

not-logged-in-f6fac0763296d5759c01    0.000174
shirogane                             0.001387
Dynamatt                              0.000676
Camelron                              0.000200
not-logged-in-7e36d1f05e1fdd4c2b64    0.001526
shocko61                              0.001773
kcarte13                              0.000273
bldelacr                              0.000161
justsomenoodles                       0.000222
jmfranci                              0.001926
wrxtminecrafter                       0.001264
not-logged-in-4ec54f98b3c226b9b959    0.012121
not-logged-in-9a813f8c400d5fedde5d    0.008429
not-logged-in-ecd063175905f82056dc    0.000618
not-logged-in-f46775ec80a9b6e163ce    0.002417
BirdLeaf                              0.007870
not-logged-in-314d48142a3a122ba666    0.005076
not-logged-in-f8c2f88355fba4282271    0.001981
clairedeu                             0.000310
pangeli5                              0.000228
not-logged-in-3569dcff4dbd0ee90f50    0.345032
crush202020  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())